In [2]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt

%matplotlib tk

In [3]:
#load data
# file path of csv file
Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_B_tstamp.csv'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'

# create dataframe
df_ = pd.read_csv(Location, sep=',')

In [4]:
df_.head()

,T[s],sppa0001_21_Timestamp_Unit0,T[s].1,sppa0001_31_Timestamp_Unit0,T[s].2,sppa0001_41_Timestamp_Unit0,T[s].3,sppa0001_22_Timestamp_Unit0,T[s].4,sppa0001_32_Timestamp_Unit0,T[s].5,sppa0001_42_Timestamp_Unit0,T[s].6,sppa0001_23_Timestamp_Unit0,T[s].7,sppa0001_33_Timestamp_Unit0,T[s].8,sppa0001_43_Timestamp_Unit0
0,0.72016,NaN,0.40120,NaN,0.63056,NaN,0.72208,NaN,2.42496,NaN,2.38420,NaN,1.93980,NaN,2.40464,NaN,2.40632,NaN
1,1.65416,NaN,1.50124,NaN,1.41580,NaN,1.89324,NaN,2.43504,NaN,2.40232,NaN,2.43908,NaN,2.42564,NaN,2.43504,NaN
2,2.40396,NaN,1.55740,NaN,1.86688,NaN,2.42168,NaN,2.44364,NaN,2.45120,NaN,2.44364,NaN,2.43300,NaN,2.43964,NaN
3,2.41688,NaN,2.37344,NaN,2.37936,NaN,2.44660,NaN,2.45520,NaN,2.45668,NaN,2.44676,NaN,2.43584,NaN,2.45540,NaN
4,2.42164,NaN,2.37684,NaN,2.39580,NaN,2.46464,NaN,2.46052,NaN,2.46228,NaN,2.45000,NaN,2.44212,NaN,2.46232,NaN


In [5]:
data = df_.iloc[:, 2].dropna()

centers = []
for elm in data:
    centers.append(int(elm*25000))

In [6]:
#load data
# file path of csv file
Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

C:\Users\33614\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

xminnoise = int(np.round(11114*(fs/1000)))
xmaxnoise = int(np.round(18511*(fs/1000)))

noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

xminspike = int(np.round(130826*(fs/1000)))
xmaxspike = int(np.round(131699*(fs/1000)))

burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

C:\Users\33614\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [8]:
#record spike

def record_spikes(signal, fs, spike_centers,
                  t_before = 0.001,
                  t_after = 0.002):
    
    data = np.array([[float(x) for x in range(int(np.round(fs*(t_before+t_after)))+1)]])
    
    for center in spike_centers:
        if center < int(np.round(fs*t_before)):
            spike = [0 for i in range(0, int(np.round(fs*t_before-center)))]
            #print('spike before concat:'+ str(len(spike)))
            #spike = np.concatenate(spike, signal.iloc[:center + int(np.round(fs*t_after))].values())
            spike = np.concatenate(spike, signal.values[:center + int(np.round(fs*t_after))])
            #print('spike after concat:'+str(len(spike)))
            data = np.insert(data, len(data), spike, axis=0)
            
        elif center > int(np.round(len(signal)-fs*t_after)):
            #spike = signal.iloc[int(np.round(center-fs*t_before)):].values()
            spike = signal.values[int(np.round(center-fs*t_before)):]
            #print('spike before concat:'+str(len(spike)))
            spike = np.concatenate(((spike,[0 for i in range(0, int(np.round(fs*t_after-(len(signal)-center)))+1)])))
            #print('spike after concat:'+str(len(spike)))
            data = np.insert(data, len(data), spike, axis=0)
            
        else :
            #spike = signal.iloc[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1].values()
            spike = signal.values[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1]
            #print(spike)
            data = np.insert(data, len(data), spike, axis=0)

    print(np.shape(data))
    data = data.transpose()
    #print(data)
    spike_data = pd.DataFrame(data)
    
    return spike_data

In [9]:
spike_data = record_spikes(filtereddf, fs, centers, 
                  t_before = 0.002, 
                  t_after = 0.002)

ValueError: could not broadcast input array from shape (1,15757) into shape (1,101)

In [ ]:
plt.figure(figsize = (100,20))
#plt.plot(spike_data)
spike_data.iloc[:,:20].plot()
plt.xlabel('Time')
plt.grid()